In [133]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from dotenv import load_dotenv

In [134]:
load_dotenv()

True

In [135]:
connection = mysql.connector.connect(
    host = os.getenv('DB_HOST'),
    user = os.getenv('DB_USER'),
    password = os.getenv('DB_PASSWORD'),
    database = os.getenv('DB_DATABASE')
)

In [136]:
cursor = connection.cursor()

In [137]:
df1 = pd.read_csv("./20042023.csv", sep=";")

In [138]:
df1['data_bloqueio'] = pd.to_datetime(df1['data_bloqueio'], format='%d/%m/%Y %H:%M')
df1['data_arquivo'] = pd.to_datetime(df1['data_arquivo'], format='%d/%m/%Y %H:%M')
df1['data_inicio_bloqueio'] = pd.to_datetime(df1['data_inicio_bloqueio'], format='%d/%m/%Y %H:%M')

In [139]:
df1['data_bloqueio'] = df1['data_bloqueio'].dt.date
df1['data_arquivo'] = df1['data_arquivo'].dt.date
df1['data_inicio_bloqueio'] = df1['data_inicio_bloqueio'].dt.date

In [140]:
df1['telefone'] = df1['telefone'].astype(object)  

In [125]:
df1

,origem,telefone,data_bloqueio,data_arquivo,empresa,data_inicio_bloqueio
0,PROCON AL,8200000000,2021-04-09,2023-04-20,TODAS,2021-05-09
1,PROCON AL,8201121001,2017-03-03,2023-04-20,TODAS,2017-04-02
2,PROCON AL,8201234567,2012-10-05,2023-04-20,TODAS,2012-11-04
3,PROCON AL,8206434118,2019-09-02,2023-04-20,TODAS,2019-10-02
4,PROCON AL,8208006000,2010-09-16,2023-04-20,TODAS,2010-10-16
...,...,...,...,...,...,...
1048570,PROCON PR,41999188008,2009-09-07,2023-04-20,TODAS,2009-10-07
1048571,PROCON PR,41999188027,2017-03-08,2023-04-20,TODAS,2017-04-07
1048572,PROCON PR,41999188044,2019-06-25,2023-04-20,TODAS,2019-07-25
1048573,PROCON PR,41999188077,2021-07-21,2023-04-20,TODAS,2021-08-20


In [141]:
df = pd.DataFrame(df1)

In [142]:
df['data_processamento'] = '2023-06-19'

In [143]:
df.columns = df.columns.str.upper()

In [144]:
df.dtypes

ORIGEM                  object
TELEFONE                object
DATA_BLOQUEIO           object
DATA_ARQUIVO            object
EMPRESA                 object
DATA_INICIO_BLOQUEIO    object
DATA_PROCESSAMENTO      object
dtype: object

In [145]:
df

,ORIGEM,TELEFONE,DATA_BLOQUEIO,DATA_ARQUIVO,EMPRESA,DATA_INICIO_BLOQUEIO,DATA_PROCESSAMENTO
0,PROCON AL,8200000000,2021-04-09,2023-04-20,TODAS,2021-05-09,2023-06-19
1,PROCON AL,8201121001,2017-03-03,2023-04-20,TODAS,2017-04-02,2023-06-19
2,PROCON AL,8201234567,2012-10-05,2023-04-20,TODAS,2012-11-04,2023-06-19
3,PROCON AL,8206434118,2019-09-02,2023-04-20,TODAS,2019-10-02,2023-06-19
4,PROCON AL,8208006000,2010-09-16,2023-04-20,TODAS,2010-10-16,2023-06-19
...,...,...,...,...,...,...,...
1048570,PROCON PR,41999188008,2009-09-07,2023-04-20,TODAS,2009-10-07,2023-06-19
1048571,PROCON PR,41999188027,2017-03-08,2023-04-20,TODAS,2017-04-07,2023-06-19
1048572,PROCON PR,41999188044,2019-06-25,2023-04-20,TODAS,2019-07-25,2023-06-19
1048573,PROCON PR,41999188077,2021-07-21,2023-04-20,TODAS,2021-08-20,2023-06-19


In [150]:
chunksize = 300000

# Particione o DataFrame em chunks de 300 registros
chunks = [df[i:i+chunksize] for i in range(0, len(df), chunksize)]

In [ ]:
# Itere sobre os chunks do DataFrame
for chunk in chunks:
    for indice, row in chunk.iterrows():
        # Obtenha os valores de cada coluna para a linha atual
        origem = row['ORIGEM']
        telefone = row['TELEFONE']
        data_bloqueio = row['DATA_BLOQUEIO']
        data_arquivo = row['DATA_ARQUIVO']
        empresa = row['EMPRESA']
        OrigemPedido = row['DATA_INICIO_BLOQUEIO']
        data_inicio_bloqueio = row['DATA_PROCESSAMENTO']

        # Execute a consulta SQL para inserir os dados na tabela
        consulta = "INSERT IGNORE INTO crm_nao_perturbe (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        valores = (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio)
        cursor.execute(consulta, valores)


In [ ]:
# Confirme as alterações e feche a conexão
conexao.commit()
conexao.close()